In [1]:
import pandas as pd
import numpy as np

# Define the schema for each file
account_schema = ['id', 'name', 'timestamp', 'black']
card_schema = ['no', 'name', 'timestamp', 'black']
acc2acc_schema = ['source_id', 'target_id', 'timestamp', 'amt', 'strategy_name', 'trade_no', 'buscode', 'trade_flag', 'anticheat_prop_cluster', 'anticheat_prop_freq', 'anticheat_prop_relation', 'anticheat_prop_action', 'anticheat_prop_buyerseller']
acc2card_schema = ['source_id', 'target_id', 'timestamp', 'amt', 'strategy_name', 'trade_no', 'buscode', 'trade_flag', 'anticheat_prop_cluster', 'anticheat_prop_freq', 'anticheat_prop_relation', 'anticheat_prop_action', 'anticheat_prop_buyerseller']

# Replace 'your_file' with the path to the file you want to read
account_raw = pd.read_csv('data/account', names=account_schema)
card_raw = pd.read_csv('data/card', names=card_schema)
acc2acc_raw = pd.read_csv('data/account_to_account', names=acc2acc_schema)
acc2card_raw = pd.read_csv('data/account_to_card', names=acc2card_schema)

In [2]:
acc2acc = acc2acc_raw[['source_id', 'target_id', 'amt', 'strategy_name', 'buscode']].copy()
acc2card = acc2card_raw[['source_id', 'target_id', 'amt', 'strategy_name', 'buscode']].copy()

name2id = {
    "Mike": 0,
    "John": 1,
    "Jobs": 2,
}

acc2acc['amt'] = acc2acc_raw['amt'].round().astype(np.uint8)
acc2card['amt'] = acc2card_raw['amt'].round().astype(np.uint8)
acc2acc['strategy_name'] = acc2acc['strategy_name'].str[-1].astype(np.uint8)
acc2card['strategy_name'] = acc2card['strategy_name'].str[-1].astype(np.uint8)
acc2acc['buscode'] = acc2acc['buscode'].str[-1].astype(np.uint8)
acc2card['buscode'] = acc2card['buscode'].str[-1].astype(np.uint8)
acc2acc['from_name'] = acc2acc['source_id'].map(account_raw.set_index('id')['name']).map(name2id).astype(np.uint8)
acc2acc['to_name'] = acc2acc['target_id'].map(account_raw.set_index('id')['name']).map(name2id).astype(np.uint8)
acc2card['from_name'] = acc2card['source_id'].map(account_raw.set_index('id')['name']).map(name2id).astype(np.uint8)
acc2card['to_name'] = acc2card['target_id'].map(card_raw.set_index('no')['name']).map(name2id).astype(np.uint8)
# acc2acc.drop(['source_id', 'target_id'], axis=1, inplace=True)
# acc2card.drop(['source_id', 'target_id'], axis=1, inplace=True)
edges = pd.concat([acc2acc, acc2card], ignore_index=True)
edges

,source_id,target_id,amt,strategy_name,buscode,from_name,to_name
0,284452,734522,0,2,2,1,1
1,734522,284452,50,1,2,1,1
2,785045,318621,9,2,2,2,1
3,318621,785045,11,1,3,1,2
4,785045,318621,3,3,3,2,1
...,...,...,...,...,...,...,...
9420698,455846,6902,0,4,1,0,1
9420699,58995,391212,8,5,3,1,0
9420700,58995,391212,0,4,2,1,0
9420701,58995,391212,0,5,2,1,0


In [3]:
id_removed = ['amt', 'strategy_name', 'buscode', 'from_name', 'to_name']
counts = edges[id_removed].groupby(id_removed).size().reset_index(name='count')
counts = counts[counts['count'] >= 10000].sort_values('count', ascending=False)
filtered_edges = pd.merge(counts, edges, on=id_removed, how='inner')[edges.columns.tolist()]
filtered_edges

,source_id,target_id,amt,strategy_name,buscode,from_name,to_name
0,182222,292213,0,3,3,0,0
1,93449,83742,0,3,3,0,0
2,268168,322613,0,3,3,0,0
3,298123,20028,0,3,3,0,0
4,423878,10744,0,3,3,0,0
...,...,...,...,...,...,...,...
4557380,570262,757190,10,3,1,0,0
4557381,544974,298429,10,3,1,0,0
4557382,120937,627917,10,3,1,0,0
4557383,744909,122241,10,3,1,0,0


In [45]:
len(counts)

167

In [70]:
import pickle
# preprocess the edges collections
for i in range(len(counts)):
    edges = filtered_edges.iloc[counts['count'].iloc[:i].sum():counts['count'].iloc[:i+1].sum()].groupby('source_id')['target_id'].apply(set).to_dict()
    count = counts.iloc[i]
    with open(f'preprocessed_edges/{count.amt}_{count.strategy_name}_{count.buscode}_{count.from_name}_{count.to_name}.pkl'.format(i), 'wb') as f:
        pickle.dump(edges, f)
    print(f'finished count {i+1}')

finished count 1
finished count 2
finished count 3
finished count 4
finished count 5
finished count 6
finished count 7
finished count 8
finished count 9
finished count 10
finished count 11
finished count 12
finished count 13
finished count 14
finished count 15
finished count 16
finished count 17
finished count 18
finished count 19
finished count 20
finished count 21
finished count 22
finished count 23
finished count 24
finished count 25
finished count 26
finished count 27
finished count 28
finished count 29
finished count 30
finished count 31
finished count 32
finished count 33
finished count 34
finished count 35
finished count 36
finished count 37
finished count 38
finished count 39
finished count 40
finished count 41
finished count 42
finished count 43
finished count 44
finished count 45
finished count 46
finished count 47
finished count 48
finished count 49
finished count 50
finished count 51
finished count 52
finished count 53
finished count 54
finished count 55
finished count 56
f

In [76]:
def find_series(edges_1, edges_2, edges_3):
    series = []
    for node1, node2_set in edges_1.items():
        for node2 in node2_set:
            node3s = edges_2.get(node2, [])
            for node3 in node3s:
                if node3 != node1:
                    for node4 in edges_3.get(node3, []):
                        if node4 != node1 and node4 != node2:
                            series.append((node1, node2, node3, node4))
                            if len(series) >= 10000:
                                return True, series
    return False, []
for i in range(len(counts)):
    edges_1 = pickle.load(open(f'preprocessed_edges/{counts.iloc[i].amt}_{counts.iloc[i].strategy_name}_{counts.iloc[i].buscode}_{counts.iloc[i].from_name}_{counts.iloc[i].to_name}.pkl', 'rb'))
    for j in range(len(counts)):
        edges_2 = pickle.load(open(f'preprocessed_edges/{counts.iloc[j].amt}_{counts.iloc[j].strategy_name}_{counts.iloc[j].buscode}_{counts.iloc[j].from_name}_{counts.iloc[j].to_name}.pkl', 'rb'))
        for k in range(len(counts)):
            edges_3 = pickle.load(open(f'preprocessed_edges/{counts.iloc[k].amt}_{counts.iloc[k].strategy_name}_{counts.iloc[k].buscode}_{counts.iloc[k].from_name}_{counts.iloc[k].to_name}.pkl', 'rb'))
            found, series = find_series(edges_1, edges_2, edges_3)
            if found:
                print(f'found {len(series)} series in {i}, {j}, {k}')
                with open(f'found_series.txt', 'a') as f:
                    f.write(f'{i},{j},{k}\n')

found 10000 series in 0, 0, 0
found 10000 series in 0, 0, 1
found 10000 series in 0, 0, 2
found 10000 series in 0, 0, 3
found 10000 series in 0, 0, 4
found 10000 series in 0, 0, 5
found 10000 series in 0, 0, 7
found 10000 series in 0, 0, 9
found 10000 series in 0, 0, 10
found 10000 series in 0, 0, 12
found 10000 series in 0, 0, 13
found 10000 series in 0, 0, 14
found 10000 series in 0, 0, 15
found 10000 series in 0, 0, 20
found 10000 series in 0, 0, 21
found 10000 series in 0, 0, 30
found 10000 series in 0, 0, 31
found 10000 series in 0, 0, 35
found 10000 series in 0, 0, 36
found 10000 series in 0, 0, 37
found 10000 series in 0, 0, 38
found 10000 series in 0, 0, 41
found 10000 series in 0, 0, 58
found 10000 series in 0, 0, 59
found 10000 series in 0, 0, 60
found 10000 series in 0, 0, 62
found 10000 series in 0, 1, 0
found 10000 series in 0, 1, 1
found 10000 series in 0, 1, 2
found 10000 series in 0, 1, 3
found 10000 series in 0, 1, 4
found 10000 series in 0, 1, 5
found 10000 series in 

KeyboardInterrupt: 

In [119]:
from pydantic import BaseModel

class Node(BaseModel):
    node_id: int
    name: str
    
class Edge(BaseModel):
    source_id_node: int
    target_id_node: int
    amt: float
    strategy_name: str
    buscode: str
    
class Pattern(BaseModel):
    nodes: list[Node]
    edges: list[Edge]
    
    # make this json serializable
    def json(self):
        return {
            'nodes': [node.model_dump_json() for node in self.nodes],
            'edges': [edge.model_dump_json() for edge in self.edges],
        }

In [120]:
found_series = pd.read_csv('found_series.txt', header=None)

id2name = {
    "Mike": 0,
    "John": 1,
    "Jobs": 2,
}

patterns = []
for c in found_series.values:
    pattern = Pattern(nodes=[], edges=[])
    pattern.nodes.append(Node(node_id=0, name=list(name2id.keys())[counts.iloc[c[0]].from_name]))
    for i in range(3):
        edge = counts.iloc[c[i]]
        pattern.edges.append(Edge(source_id_node=i, target_id_node=i+1, amt=float(edge.amt), strategy_name=f'{edge.strategy_name}', buscode=f'{edge.buscode}'))
        node = Node(node_id=i+1, name=list(name2id.keys())[edge.to_name])
        pattern.nodes.append(node)
    patterns.append(pattern)

In [121]:
import json
with open('bdci_data.json', 'w') as f:
    json.dump([p.model_dump() for p in patterns], f, indent=4)